# Imports

In [182]:
# %pylab inline
import pandas as pd
import numpy as np
import json
import sys
import folium
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import cross_validation
from sklearn import metrics

# Read Data

In [31]:
file = input('Enter path for training dataset: ')
train_data = pd.read_csv(file,encoding='ISO-8859-1')
train_data
file = input('Enter path for new dataset: ')
new_data = pd.read_csv(file, encoding='ISO-8859-1')

Enter path for training dataset: database.csv
Enter path for new dataset: tweets May 26.csv


# Train Model

In [170]:
X = train_data
y = train_data['classification 2']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1, random_state=1)

vect = CountVectorizer(ngram_range=(1,5), max_df=0.99)

X_train_dtm = vect.fit_transform(X_train.text)
X_test_dtm = vect.transform(X_test.text)

X_tokens = vect.get_feature_names()
# print (X_tokens[2040:])

vect = CountVectorizer(stop_words = 'english', ngram_range=(1,5), max_df=0.99)

X_train_dtm = vect.fit_transform(X_train.text)
X_test_dtm = vect.transform(X_test.text)

nb = MultinomialNB(alpha=0.001)
nb = nb.fit(X_train_dtm, y_train)

y_predicted = nb.predict(X_test_dtm)

print(confusion_matrix(y_test, y_predicted))
print(classification_report(y_test, y_predicted))
print('Accuracy score: '+str(metrics.accuracy_score(y_test, y_predicted)))

[[15  6]
 [ 4 54]]
             precision    recall  f1-score   support

      Bully       0.79      0.71      0.75        21
  Not bully       0.90      0.93      0.92        58

avg / total       0.87      0.87      0.87        79

Accuracy score: 0.873417721519


# Show Predictions

In [150]:
X_test_dtm = vect.transform(new_data['text'])
y_predicted = nb.predict(X_test_dtm)

input_text = pd.DataFrame(new_data.text.tolist())
prediction = pd.DataFrame(y_predicted)
table = pd.concat([input_text,prediction],axis=1)
table.columns = ['Input','Prediction']
table.to_csv('predicted_'+file)
table.sample(n=20)

,Input,Prediction
158,taas taas nagd ni tolerance ka roommates ko sa...,Not bully
30233,Team @MercadoJen all d way <ed><U+00A0><U+00BD...,Not bully
1099,I scream for ice cream and coffee <ed><U+00A0>...,Not bully
126487,18 sec then 12 sec. Another 30sec kiss. Grabe ...,Not bully
129871,masakit lang talaga sa heart ko di ko na kaya ...,Not bully
22043,SG is real na. Booking nalang tsaka confirmation,Not bully
114607,Sleeping over for the night. (@ EM's Barrio So...,Not bully
78072,anyone knows how to make a mood board? i need ...,Not bully
100300,"@jaypriet0 I love it, feels so cool.",Not bully
41838,#ALDUBxDTBYFinale road to 600k na dotdot pa,Not bully


# Tokens Influence

In [152]:
X_tokens = vect.get_feature_names()

bully_count = nb.feature_count_[0, :].tolist()
not_bully_count = nb.feature_count_[1, :].tolist()

cols = ['tokens','Bully','Not bully']
num = 0
data = []
while num < len(X_tokens):
    data.append([X_tokens[num],bully_count[num],not_bully_count[num]])
    num += 1
tokens = pd.DataFrame(data=data,columns=cols)
tokens.sort_values('Bully', ascending=False)[:50]

,tokens,Bully,Not bully
4107,ed,134.0,688.0
4165,ed ed,94.0,494.0
8295,ka,79.0,53.0
12128,mo,70.0,35.0
4211,ed ed ed,54.0,300.0
5464,gago,51.0,53.0
19200,tangina,43.0,68.0
4220,ed ed ed ed,39.0,225.0
2803,co,38.0,119.0
7118,https co,38.0,119.0


# Mapping of Bully Tweets

In [188]:
new_data = pd.read_csv('tweets May 23.csv',encoding='ISO-8859-1')

new_data = new_data[np.isfinite(new_data['lat'])]

X_test_dtm  = vect.transform(new_data['text'])

y_predicted = nb.predict(X_test_dtm)

lat = new_data.lat.tolist()
lon = new_data.lon.tolist()
name = new_data.screen_name.tolist()

input_text = pd.DataFrame(tweet_text)
prediction = pd.DataFrame(y_predicted)
name = pd.DataFrame(name)
lat = pd.DataFrame(lat)
lon = pd.DataFrame(lon)

tweet_dots = pd.concat([lat,lon,name,prediction,input_text],axis=1)
tweet_dots.columns = ['lat','lon','name','Prediction','text']
# table.to_csv('predicted_'+file)
# table.sample(n=20)

# #tweet_dots = pd.concat([lat_df,lon_df],axis=1)

tweet_dots['Prediction']
coord = tweet_dots[tweet_dots['Prediction']=='Bully']

# # coord
# # #map_object = folium.Map(location=[65, -18.6], zoom_start=6, tiles="Stamen toner")
map_object = folium.Map(location=[14.5995, 120.9842], zoom_start=12)# 14.5995° N, 120.9842° E Manila

# # #marker = folium.features.CircleMarker([(14.5995,120.8889),(14.5980,120.8889),radius=3,color="red",fill_color='red', popup="sample tweets")

for lat,lon,name in zip(coord['lat'],coord['lon'],coord['name']):
    map_object.add_child(folium.Marker(location=[lat,lon], popup=(folium.Popup(name))))

# # # marker = folium.features.Marker(location=[coord['lat'][1],coord['lon'][1]], popup="sample tweets")
# # # map_object.add_child(marker)

folium.Map.save(map_object, "index.html")